[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/05_Devices_and_HumanInLoop.ipynb)

# Sensor/Pump Models and Human-in-the-loop

**Goal:** simulate CGM noise, pump limits, and intervention callbacks.


In [ ]:
from __future__ import annotations
from pathlib import Path
import os
import sys
import subprocess


def _find_repo_root() -> Path | None:
    for root in [Path.cwd(), *Path.cwd().parents]:
        if (root / "pyproject.toml").exists() and (root / "src").exists():
            return root
    return None

repo_root = _find_repo_root()
if repo_root is None:
    try:
        import google.colab  # type: ignore
        in_colab = True
    except Exception:
        in_colab = False

    if not in_colab:
        raise RuntimeError("Run this notebook inside the IINTS-SDK repo or on Colab.")

    if not Path("IINTS-SDK").exists():
        subprocess.check_call(["git", "clone", "https://github.com/python35/IINTS-SDK.git"])
    repo_root = Path("IINTS-SDK").resolve()

os.chdir(repo_root)
sys.path.insert(0, str(repo_root / "src"))
print("Repo root:", repo_root)


In [ ]:
from iints.validation import load_patient_config_by_name
from iints.core.simulator import Simulator
from iints.core.algorithms.fixed_basal_bolus import FixedBasalBolus
from iints.core.patient.models import PatientModel
from iints.core.devices.models import SensorModel, PumpModel
from iints.core.safety import InputValidator

patient_config = load_patient_config_by_name("clinic_safe_baseline").model_dump()
patient_config.update({"glucose_decay_rate": 0.01, "basal_insulin_rate": 0.4, "initial_glucose": 150.0})

patient = PatientModel(**patient_config)
algorithm = FixedBasalBolus(settings={"fixed_basal_rate": 0.4, "carb_ratio": 12.0})

sensor = SensorModel(noise_std=3.0, lag_minutes=5, dropout_prob=0.01, seed=1)
pump = PumpModel(max_units_per_step=0.6, dropout_prob=0.01, seed=1)

def on_step(context):
    # Example intervention: give carbs if glucose drops too low
    if context["glucose_actual_mgdl"] < 75:
        return {"additional_carbs": 10}
    return None

sim = Simulator(
    patient_model=patient,
    algorithm=algorithm,
    time_step=5,
    seed=1,
    sensor_model=sensor,
    pump_model=pump,
    on_step=on_step,
)

# Relax ROC validation slightly for device/noise demo
sim.input_validator = InputValidator(max_glucose_delta_per_5_min=60.0)

results, safety = sim.run_batch(180)
results[["time_minutes", "glucose_actual_mgdl", "sensor_status", "pump_status"]].head()


### Recap
You now have realistic device noise + intervention hooks.
